In [1]:
# export dependencies' file paths
import sys 
sys.path.append('/home/jupyter/.local/lib/python3.10/site-packages')
sys.path.append('/usr/lib/python3/dist-packages')
sys.path.append('/bin/google-chrome')
sys.path.append('/bin')

In [2]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
import urllib
from datetime import datetime
import os
import time
from urllib.parse import urlencode, urljoin

In [3]:
# used to avoid Amazon blocking the scraper 
SCRAPEOPS_API_KEY = 'ced5bffb-0941-4ba2-8faa-ee55f55f1f0a'

In [4]:
def scrapeops_url(url):
    payload = {'api_key': SCRAPEOPS_API_KEY, 'url': url, 'country': 'us'}
    proxy_url = 'https://proxy.scrapeops.io/v1/?' + urlencode(payload)
    return proxy_url

In [8]:
def amazon_image_scrape(name_item, topic_name):
    search_url = f'https://www.amazon.com/s?k={topic_name}&page=1'

    html = requests.get(scrapeops_url(search_url))
    
    print(html)
    soup = BeautifulSoup(html.content, features="lxml")
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    arr_list = [] # store searched result links

    df = pd.DataFrame([], columns=['Title', 'Source URL', 'Time', 'Source']) # create data frame

    for link in links:
        arr_list.append(link.get('href'))

    for count, link in enumerate(arr_list):
        webpage_url = "https://www.amazon.com" + link
        webpage = requests.get(webpage_url, headers=HEADERS)
        new_soup = BeautifulSoup(webpage.content, "lxml")
        
        title = None
        
        # get product title
        try:
            title = new_soup.find("span", attrs={"id": 'productTitle'}).text.strip()
        except AttributeError:
            print("Cannot retrieve product title")

        current_time = datetime.now()
        formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")

        row = {
            "Title": title,
            "Source URL": webpage_url,
            "Time": formatted_time,
            "Source": "Amazon"
        }

        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

        images = re.findall('"hiRes":"(.+?)"', webpage.text)

        for num, img_url in enumerate(images):
            img_dir = "./img/{topic}/".format(topic=name_item)
            os.makedirs(img_dir, exist_ok=True)  # create the directory if it doesn't exist
            img_filename = img_dir + "image{num}_amazon.jpg".format(num=num)
            img_filename.format(topic=name_item, i=num)
            urllib.request.urlretrieve(img_url, img_filename) # retrieve images

    return df

In [9]:
output_amazon_df = amazon_image_scrape("toys", "toys")
display(output_amazon_df)

<Response [403]>


,Title,Source URL,Time,Source


In [10]:
gg

NameError: name 'gg' is not defined